Common definitions

In [ ]:
sys.path.append( '..' )

from enum import IntEnum
class FRAME_SIZE_E(IntEnum):
    SMALL = 1
    TYPICAL = 2
    WIDE = 3
    TALL = 4
    LARGE = 5
    NUM_FRAMES_SIZES = 6


Vanilla Python Class example

In [ ]:
class pythonParams:
    #Constructor
    def __init__(self, FRAME_COUNT=5, 
                       FRAME_WIDTH=640, 
                       FRAME_HEIGHT=480, 
                       MEM_REGIONS=None,
                       FRAME_SIZE="SMALL"):
        self.FRAME_COUNT = FRAME_COUNT
        self.FRAME_WIDTH = FRAME_WIDTH
        self.FRAME_HEIGHT = FRAME_HEIGHT
        #Convert string to enum type
        self.FRAME_SIZE = FRAME_SIZE_E[FRAME_SIZE]
        #Mutatble defaults, such as lists, have to be handled carefully
        self.MEM_REGIONS = MEM_REGIONS if MEM_REGIONS is not None else ["RAM"]

    #string class representation
    def __repr__(self):
        return f"pythonParams - " + \
                f"FRAME_COUNT: {self.FRAME_COUNT}, " + \
                f"FRAME_WIDTH: {self.FRAME_WIDTH}, " + \
                f"FRAME_HEIGHT: {self.FRAME_HEIGHT}, " + \
                f"FRAME_SIZE: {self.FRAME_SIZE.name}, " + \
                f"MEM_REGIONS: {self.MEM_REGIONS}"

In [ ]:
params = pythonParams(FRAME_WIDTH=200, FRAME_SIZE="LARGE")
print(params)

dataclasses class example

No more manually creating \_\_init\_\_ and \_\_repr\_\_ functions

\_\_post_init\_\_ is used to add custom \_\_init\_\_ behavior

In [ ]:
from dataclasses import dataclass, field

@dataclass
class dataclassesParams:
    FRAME_COUNT  : int = field(default=5)
    FRAME_WIDTH  : int = 640 # dataclass implies field(default=val)
    FRAME_HEIGHT : int = field(default=480)
    FRAME_SIZE   : FRAME_SIZE_E = field(default="SMALL")
    #dataclasses handles mutatable defaults cleaner with factory functions
    MEM_REGIONS  : list = field(default_factory=lambda:["RAM"])
    
    def __post_init__(self):
      #Still need to convert that string input to an enum
      self.FRAME_SIZE = FRAME_SIZE_E[self.FRAME_SIZE]

In [ ]:
params = dataclassesParams(FRAME_WIDTH=200, FRAME_SIZE="LARGE")
print(params)

DatagenDV class example

In [ ]:
import datagenDV as dg
from dataclasses import dataclass

@dataclass
class datagenParams(dg.ParamsBase):
    FRAME_COUNT  : int = dg.field(5)
    FRAME_WIDTH  : int = dg.field(640)
    FRAME_HEIGHT : int = dg.field(480)
    FRAME_SIZE   : FRAME_SIZE_E = dg.field("SMALL")
    MEM_REGIONS  : list = dg.field(lambda:["RAM"])
    
    
    
    

In [ ]:
params = datagenParams(FRAME_WIDTH=200, FRAME_SIZE="LARGE")
print(params)

datagenDV also provides runtime warnings for bad types.

In [ ]:
try:
    params = datagenParams(FRAME_WIDTH="Not an integer")
except Exception as e:
    print(e)

YAML input/output

In [ ]:
@dataclass
class DatagenParams(dg.YAMLParamsBase):
    #YAML fields
    FRAME_COUNT  : int   = dg.field(5, dir='in_out')
    FRAME_WIDTH  : int   = dg.field(640, dir='in_out')
    FRAME_HEIGHT : int   = dg.field(480, dir='in_out')
    MEM_REGIONS  : list  = dg.field(lambda:["RAM"], dir='in')
    FRAME_SIZE   : FRAME_SIZE_E  = dg.field("SMALL", dir='in_out')
    ADDRESS : int = dg.field(None, dir='out')
    
    

In [ ]:
input_yaml = """
param_example: !DatagenParams
    FRAME_COUNT: 3
    FRAME_HEIGHT: 320
    MEM_REGIONS: 
        - ROM
        - RAM 
    FRAME_SIZE : LARGE
"""

In [ ]:
from ruamel.yaml import YAML
yaml = YAML()
yaml.register_class(DatagenParams)
data = yaml.load(input_yaml)
data['param_example'].ADDRESS = 1234
yaml.dump(data, sys.stdout, transform=lambda s: s.replace('!', ''))
#FIXME - figure out the best way to allow !class name on dump optionally

In [ ]:
"""
param_example: 
  FRAME_COUNT: 3
  FRAME_WIDTH: 640
  FRAME_HEIGHT: 320
  FRAME_SIZE: LARGE
  ADDRESS: 1234
"""

PyVSC Constrained Randomization

In [ ]:
import vsc
@dg.rand_dataclass
class FrameParams(dg.YAMLParamsBase):
    FRAME_SIZE : int = dg.rand_field(vsc.rand_bit_t, 64)
    FRAME_WIDTH  : int = dg.rand_field(vsc.rand_bit_t, 32)
    FRAME_HEIGHT : int = dg.rand_field(vsc.rand_bit_t, 32)


In [ ]:
inst = FrameParams()
inst.randomize()
print(inst)

Add some constraints

In [ ]:
@dg.rand_dataclass
class FrameParams(dg.YAMLParamsBase):
    FRAME_SIZE   : int = dg.rand_field(vsc.rand_bit_t, 64)
    FRAME_WIDTH  : int = dg.rand_field(vsc.rand_bit_t, 32)
    FRAME_HEIGHT : int = dg.rand_field(vsc.rand_bit_t, 32)
    
    @vsc.constraint
    def frame_dimensions_c(self):
        self.rand_FRAME_SIZE == self.rand_FRAME_WIDTH * self.rand_FRAME_HEIGHT

    @vsc.constraint
    def sizes_c(self):
        self.rand_FRAME_SIZE.inside(vsc.rangelist((16, 65536)))
        self.rand_FRAME_WIDTH.inside(vsc.rangelist((4, 256)))
        self.rand_FRAME_HEIGHT.inside(vsc.rangelist((4, 256)))


In [ ]:
inst = FrameParams()
inst.randomize()
print(inst)

In [ ]:
inst = FrameParams(FRAME_WIDTH=100)
inst.randomize()
print(inst)

What if you want to check your random distribution? Python makes it easy with matplotlib


In [ ]:
import matplotlib.pyplot as plt
def randomize_and_graph_histograms(rand_class, N=10000):
    class_inst = rand_class()
    var_list = {}
    #Randomize N times, collecting values for graphing
    for i in range(0,N):
        class_inst.randomize()
        class_inst_vars =  vars(class_inst)
        for v in class_inst_vars:
            if v.startswith("rand"):
                var_list.setdefault(v, []).append(class_inst_vars[v].get_val())
  
    fig, axs = plt.subplots(1,len(var_list)) 
    for i,k in enumerate(var_list.keys()):
        axs[i].hist(var_list[k], bins=500)
        axs[i].set_title(k)

    fig.set_size_inches(24, 4)
    fig.show()
      
#Call our generic function with FrameParams and randomize it 10000 times  
randomize_and_graph_histograms(FrameParams, N=1000)

Ctypes binary creation and loading